Copyright 2020 DeepMind Technologies Limited.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

<p align="center">
  <h1 align="center">TAPIR: Tracking Any Point with per-frame Initialization and temporal Refinement</h1>
  <p align="center">
    <a href="http://www.carldoersch.com/">Carl Doersch</a>
    ·
    <a href="https://yangyi02.github.io/">Yi Yang</a>
    ·
    <a href="https://scholar.google.com/citations?user=Jvi_XPAAAAAJ">Mel Vecerik</a>
    ·
    <a href="https://scholar.google.com/citations?user=cnbENAEAAAAJ">Dilara Gokay</a>
    ·
    <a href="https://www.robots.ox.ac.uk/~ankush/">Ankush Gupta</a>
    ·
    <a href="http://people.csail.mit.edu/yusuf/">Yusuf Aytar</a>
    ·
    <a href="https://scholar.google.co.uk/citations?user=IUZ-7_cAAAAJ">Joao Carreira</a>
    ·
    <a href="https://www.robots.ox.ac.uk/~az/">Andrew Zisserman</a>
  </p>
  <h3 align="center"><a href="https://arxiv.org/abs/2306.08637">Paper</a> | <a href="https://deepmind-tapir.github.io">Project Page</a> | <a href="https://github.com/deepmind/tapnet">GitHub</a> | <a href="https://github.com/deepmind/tapnet/tree/main#running-tapir-locally">Live Demo</a> </h3>
  <div align="center"></div>
</p>

<p align="center">
  <a href="">
    <img src="https://storage.googleapis.com/dm-tapnet/swaying_gif.gif" alt="Logo" width="50%">
  </a>
</p>



In [1]:
# @title Install Code and Dependencies {form-width: "25%"}
!pip install git+https://github.com/google-deepmind/tapnet.git

  Cloning https://github.com/google-deepmind/tapnet.git to /tmp/pip-req-build-s2pl0dbe
  Running command git clone --filter=blob:none --quiet https://github.com/google-deepmind/tapnet.git /tmp/pip-req-build-s2pl0dbe
  Resolved https://github.com/google-deepmind/tapnet.git to commit c7e3ff3000a53ec628ec676acc719722f637bba4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.9/373.9 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.7/515.7 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 68.1 MB/s eta 0:00:00
  Created wheel for tapnet: filename=tapnet-0.1.0-py3-none-any.whl size=205574 sha256=aee0691edd080b70f72a30f8d896e3bf15f5466a2228e5b7d25e819319e68458
  Stored in directory: /tmp/pip-

In [2]:
MODEL_TYPE = 'bootstapir'  # 'tapir' or 'bootstapir'

In [3]:
# @title Download Model {form-width: "25%"}

%mkdir tapnet/checkpoints

if MODEL_TYPE == "tapir":
  !wget -P tapnet/checkpoints https://storage.googleapis.com/dm-tapnet/causal_tapir_checkpoint.npy
else:
  !wget -P tapnet/checkpoints https://storage.googleapis.com/dm-tapnet/bootstap/causal_bootstapir_checkpoint.npy

%ls tapnet/checkpoints

mkdir: cannot create directory ‘tapnet/checkpoints’: No such file or directory
--2025-01-09 04:50:57--  https://storage.googleapis.com/dm-tapnet/bootstap/causal_bootstapir_checkpoint.npy
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.99.207, 142.250.107.207, 142.251.188.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.99.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 218818353 (209M) [application/octet-stream]
Saving to: ‘tapnet/checkpoints/causal_bootstapir_checkpoint.npy’

causal_bootstapir_c 100%[===================>] 208.68M   129MB/s    in 1.6s    

2025-01-09 04:50:59 (129 MB/s) - ‘tapnet/checkpoints/causal_bootstapir_checkpoint.npy’ saved [218818353/218818353]

causal_bootstapir_checkpoint.npy


In [4]:
# @title Imports {form-width: "25%"}
# %matplotlib widget
from google.colab import output
import jax
import matplotlib.pyplot as plt
import mediapy as media
import numpy as np
from tapnet.models import tapir_model
from tapnet.utils import model_utils
from tapnet.utils import transforms
from tapnet.utils import viz_utils

output.enable_custom_widget_manager()

ValueError: Key backend: 'module://ipympl.backend_nbagg' is not a valid value for backend; supported values are ['gtk3agg', 'gtk3cairo', 'gtk4agg', 'gtk4cairo', 'macosx', 'nbagg', 'notebook', 'qtagg', 'qtcairo', 'qt5agg', 'qt5cairo', 'tkagg', 'tkcairo', 'webagg', 'wx', 'wxagg', 'wxcairo', 'agg', 'cairo', 'pdf', 'pgf', 'ps', 'svg', 'template', 'module://ipykernel.pylab.backend_inline', 'inline']

In [ ]:
# @title Load Checkpoint {form-width: "25%"}

if MODEL_TYPE == 'tapir':
  checkpoint_path = 'tapnet/checkpoints/causal_tapir_checkpoint_panning.npy'
else:
  checkpoint_path = 'tapnet/checkpoints/causal_bootstapir_checkpoint.npy'
ckpt_state = np.load(checkpoint_path, allow_pickle=True).item()
params, state = ckpt_state['params'], ckpt_state['state']

kwargs = dict(
    use_causal_conv=True,
    bilinear_interp_with_depthwise_conv=False,
    pyramid_level=0,
)
if MODEL_TYPE == 'bootstapir':
  kwargs.update(
      dict(pyramid_level=1, extra_convs=True, softmax_temperature=10.0)
  )
tapir = tapir_model.ParameterizedTAPIR(params, state, tapir_kwargs=kwargs)

In [ ]:
# @title Build Model {form-width: "25%"}

# Internally, the tapir model has three stages of processing: computing
# image features (get_feature_grids), extracting features for each query point
# (get_query_features), and estimating trajectories given query features and
# the feature grids where we want to track (estimate_trajectories).  For
# tracking online, we need extract query features on the first frame only, and
# then call estimate_trajectories on one frame at a time.


def online_model_init(frames, query_points):
  """Initialize query features for the query points."""
  frames = model_utils.preprocess_frames(frames)
  feature_grids = tapir.get_feature_grids(frames, is_training=False)
  query_features = tapir.get_query_features(
      frames,
      is_training=False,
      query_points=query_points,
      feature_grids=feature_grids,
  )
  return query_features


online_model_init = jax.jit(online_model_init)


def online_model_predict(frames, query_features, causal_context):
  """Compute point tracks and occlusions given frames and query points."""
  frames = model_utils.preprocess_frames(frames)
  feature_grids = tapir.get_feature_grids(frames, is_training=False)
  trajectories = tapir.estimate_trajectories(
      frames.shape[-3:-1],
      is_training=False,
      feature_grids=feature_grids,
      query_features=query_features,
      query_points_in_video=None,
      query_chunk_size=64,
      causal_context=causal_context,
      get_causal_context=True,
  )
  causal_context = trajectories['causal_context']
  del trajectories['causal_context']
  # Take only the predictions for the final resolution.
  # For running on higher resolution, it's typically better to average across
  # resolutions.
  tracks = trajectories['tracks'][-1]
  occlusions = trajectories['occlusion'][-1]
  uncertainty = trajectories['expected_dist'][-1]
  visibles = model_utils.postprocess_occlusions(occlusions, uncertainty)
  return tracks, visibles, causal_context


online_model_predict = jax.jit(online_model_predict)

In [ ]:
# @title Utility Functions {form-width: "25%"}


def sample_random_points(frame_max_idx, height, width, num_points):
  """Sample random points with (time, height, width) order."""
  y = np.random.randint(0, height, (num_points, 1))
  x = np.random.randint(0, width, (num_points, 1))
  t = np.random.randint(0, frame_max_idx + 1, (num_points, 1))
  points = np.concatenate((t, y, x), axis=-1).astype(
      np.int32
  )  # [num_points, 3]
  return points

In [ ]:
# @title Load an Exemplar Video {form-width: "25%"}

%mkdir tapnet/examplar_videos

!wget -P tapnet/examplar_videos https://storage.googleapis.com/dm-tapnet/horsejump-high.mp4

video = media.read_video("tapnet/examplar_videos/horsejump-high.mp4")
media.show_video(video, fps=10)

In [ ]:
# @title Progressively Predict Sparse Point Tracks {form-width: "25%"}

resize_height = 256  # @param {type: "integer"}
resize_width = 256  # @param {type: "integer"}
num_points = 20  # @param {type: "integer"}

frames = media.resize_video(video, (resize_height, resize_width))
query_points = sample_random_points(
    0, frames.shape[1], frames.shape[2], num_points
)

# Extract features for the query point at the first frame.
query_features = online_model_init(frames[None, 0:1], query_points[None])
# If there are query points at other frames, we need to call:
# query_features = online_model_init(frames[None], query_points[None])

# We need a causal state for every PIPs resolution; therefore,
# construct_initial_causal_state's second param is the number of resolutions.
# query_features.resolutions is one more than the number of PIPs resolutions
# (since there's an 'initialization' resolution as well).
# Of course, since we're running at 256x256 in this demo, there will only be
# one PIPs resolution.
#
# Note that causal_state can also be initialized to None, but this will mean
# online_model_predict gets compiled twice.
causal_state = tapir.construct_initial_causal_state(
    query_points.shape[0], len(query_features.resolutions) - 1
)

# Predict point tracks frame by frame
predictions = []
for i in range(frames.shape[0]):
  # Note: we add a batch dimension.
  tracks, visibles, causal_state = online_model_predict(
      frames=frames[None, i : i + 1],
      query_features=query_features,
      causal_context=causal_state,
  )
  predictions.append({'tracks': tracks, 'visibles': visibles})

tracks = np.concatenate([x['tracks'][0] for x in predictions], axis=1)
visibles = np.concatenate([x['visibles'][0] for x in predictions], axis=1)

# Visualize sparse point tracks
height, width = video.shape[1:3]
tracks = transforms.convert_grid_coordinates(
    tracks, (resize_width, resize_height), (width, height)
)
video_viz = viz_utils.paint_point_track(video, tracks, visibles)
media.show_video(video_viz, fps=10)

In [ ]:
# @title Download TAPVid-DAVSIS Dataset {form-width: "25%"}
!wget https://storage.googleapis.com/dm-tapnet/tapvid_davis.zip
!unzip tapvid_davis.zip

In [ ]:
# @title DAVIS First Eval on 256x256 Resolution {form-width: "25%"}
%%time

from tapnet import evaluation_datasets

davis_dataset = evaluation_datasets.create_davis_dataset(
    'tapvid_davis/tapvid_davis.pkl', query_mode='first'
)

summed_scalars = None
for sample_idx, sample in enumerate(davis_dataset):
  sample = sample['davis']
  frames = np.round((sample['video'][0] + 1) / 2 * 255).astype(np.uint8)
  query_points = sample['query_points'][0]

  # Extract features for the query point.
  query_features = online_model_init(frames[None], query_points[None])

  causal_state = tapir.construct_initial_causal_state(
      query_points.shape[0], len(query_features.resolutions) - 1
  )

  # Predict point tracks frame by frame
  predictions = []
  for i in range(frames.shape[0]):
    # Note: we add a batch dimension.
    tracks, visibles, causal_state = online_model_predict(
        frames=frames[None, i : i + 1],
        query_features=query_features,
        causal_context=causal_state,
    )
    predictions.append({'tracks': tracks, 'visibles': visibles})

  tracks = np.concatenate([x['tracks'][0] for x in predictions], axis=1)
  visibles = np.concatenate([x['visibles'][0] for x in predictions], axis=1)
  occluded = ~visibles

  query_points = sample['query_points'][0]

  scalars = evaluation_datasets.compute_tapvid_metrics(
      query_points[None],
      sample['occluded'],
      sample['target_points'],
      occluded[None],
      tracks[None],
      query_mode='first',
  )
  scalars = jax.tree.map(lambda x: np.array(np.sum(x, axis=0)), scalars)
  print(sample_idx, scalars)

  if summed_scalars is None:
    summed_scalars = scalars
  else:
    summed_scalars = jax.tree.map(np.add, summed_scalars, scalars)

  num_samples = sample_idx + 1
  mean_scalars = jax.tree.map(lambda x: x / num_samples, summed_scalars)
  print(mean_scalars)

In [ ]:
# @title Select Any Points at Any Frame {form-width: "25%"}

select_frame = 0  # @param {type:"slider", min:0, max:49, step:1}

# Generate a colormap with 20 points, no need to change unless select more than 20 points
colormap = viz_utils.get_colors(20)

fig, ax = plt.subplots(figsize=(10, 5))
ax.imshow(video[select_frame])
ax.axis('off')
ax.set_title(
    'You can select more than 1 point. After selecting enough points, run the'
    ' next cell.'
)

select_points = []


# Event handler for mouse clicks
def on_click(event):
  if event.button == 1 and event.inaxes == ax:  # Left mouse button clicked
    x, y = int(np.round(event.xdata)), int(np.round(event.ydata))

    select_points.append(np.array([x, y]))

    color = colormap[len(select_points) - 1]
    color = tuple(np.array(color) / 255.0)
    ax.plot(x, y, 'o', color=color, markersize=5)
    plt.draw()


fig.canvas.mpl_connect('button_press_event', on_click)
plt.show()

In [ ]:
# @title Predict Point Tracks for the Selected Points {form-width: "25%"}

resize_height = 256  # @param {type: "integer"}
resize_width = 256  # @param {type: "integer"}


def convert_select_points_to_query_points(frame, points):
  """Convert select points to query points.

  Args:
    points: [num_points, 2], [t, y, x]

  Returns:
    query_points: [num_points, 3], [t, y, x]
  """
  points = np.stack(points)
  query_points = np.zeros(shape=(points.shape[0], 3), dtype=np.float32)
  query_points[:, 0] = frame
  query_points[:, 1] = points[:, 1]
  query_points[:, 2] = points[:, 0]
  return query_points


frames = media.resize_video(video, (resize_height, resize_width))
query_points = convert_select_points_to_query_points(
    select_frame, select_points
)
height, width = video.shape[1:3]
query_points = transforms.convert_grid_coordinates(
    query_points,
    (1, height, width),
    (1, resize_height, resize_width),
    coordinate_format='tyx',
)

query_features = online_model_init(frames[None], query_points[None])
causal_state = tapir.construct_initial_causal_state(
    query_points.shape[0], len(query_features.resolutions) - 1
)

# Predict point tracks frame by frame
predictions = []
for i in range(frames.shape[0]):
  # Note: we add a batch dimension.
  tracks, visibles, causal_state = online_model_predict(
      frames=frames[None, i : i + 1],
      query_features=query_features,
      causal_context=causal_state,
  )
  predictions.append({'tracks': tracks, 'visibles': visibles})

tracks = np.concatenate([x['tracks'][0] for x in predictions], axis=1)
visibles = np.concatenate([x['visibles'][0] for x in predictions], axis=1)

# Visualize sparse point tracks
tracks = transforms.convert_grid_coordinates(
    tracks, (resize_width, resize_height), (width, height)
)
video_viz = viz_utils.paint_point_track(video, tracks, visibles, colormap)
media.show_video(video_viz, fps=10)

That's it!